In [3]:
import os
import sys
import numpy as np
import pandas as pd

import rocks

rocks.set_log_level("error")

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import figure_setup as fs

# You need SKA somewhere: https://github.com/bcarry/ska
# sys.path.append("/home/bcarry/Documents/softs/")
# from ska import tools as skatools

from figure_mask import compute_mask, print_statistics, compute_mask_spin

# Get ZTF fit

In [4]:
# Local Configuration
data_fink = "../"
bft_file = os.path.join(data_fink, "data", "ssoBFT-latest.parquet")

In [5]:
# ZTF filters
filters = {"1": "g", "2": "r"}
# fink_colors = ["#15284F", "#F5622E"]
fink_colors = ["C0", "C1"]

In [6]:
data = pd.read_parquet(os.path.join(data_fink, "data", "ztf", "sso_ZTF.parquet"))

In [7]:
# bft = pd.read_parquet(bft_file)

In [8]:
# bft.columns[bft.columns.str.contains(r"^(?=.*Atlas)(?=.*orange)")]

In [9]:
# cols = [
#     "sso_number",
#     "sso_name",
#     "sso_class",
#     "orbital_elements.semi_major_axis.value",
#     "orbital_elements.eccentricity.value",
#     "orbital_elements.inclination.value",
#     "orbital_elements.node_longitude.value",
#     "orbital_elements.perihelion_argument.value",
#     "orbital_elements.mean_anomaly.value",
#     "orbital_elements.mean_motion.value",
#     "family.family_number",
#     "family.family_name",
#     "tisserand_parameters.Jupiter.value",
#     "albedo.value",
#     "absolute_magnitude.value",
#     "diameter.value",
#     "taxonomy.class",
#     "taxonomy.complex",
#     "taxonomy.waverange",
#     "taxonomy.scheme",
#     "taxonomy.technique",
#     "colors.g-r.color.value",
#     "colors.g-r.color.error.min",
#     "colors.g-r.color.error.max",
#     "colors.g-r.facility",
#     "colors.g-r.observer",
#     "colors.g-r.epoch",
#     "colors.g-r.delta_time",
#     "colors.g-r.id_filter_1",
#     "colors.g-r.id_filter_2",
#     "colors.g-r.phot_sys",
#     "colors.g-r.technique",
#     "phase_functions.Misc/Atlas.orange.H.value",
#     "phase_functions.Misc/Atlas.orange.G1.value",
#     "phase_functions.Misc/Atlas.orange.G2.value",
#     "phase_functions.Misc/Atlas.cyan.H.value",
#     "phase_functions.Misc/Atlas.cyan.G1.value",
#     "phase_functions.Misc/Atlas.cyan.G2.value",
#     "spins.1.obliquity",
#     "spins.1.RA0.value",
#     "spins.1.DEC0.value",
#     "spins.1.long.value",
#     "spins.1.lat.value",
#     "spins.1.technique",
#     "spins.2.obliquity",
#     "spins.2.RA0.value",
#     "spins.2.DEC0.value",
#     "spins.2.long.value",
#     "spins.2.lat.value",
#     "spins.2.technique",
#     "spins.3.obliquity",
#     "spins.3.RA0.value",
#     "spins.3.DEC0.value",
#     "spins.3.long.value",
#     "spins.3.lat.value",
#     "spins.3.technique",
#     "spins.4.obliquity",
#     "spins.4.RA0.value",
#     "spins.4.DEC0.value",
#     "spins.4.long.value",
#     "spins.4.lat.value",
#     "spins.4.technique",
# ]

# data = data.merge(bft[cols], left_on="name", right_on="sso_name", how="left")

In [10]:
# data.columns[data.columns.str.contains("Atlas")]

In [11]:
len(data)

122675

# Compare results

In [12]:
data.HG1G2_status.value_counts()

HG1G2_status
 2.0    113644
 4.0      6886
 1.0      1320
 3.0       521
-2.0       301
 0.0         3
Name: count, dtype: int64

In [13]:
mask_SHG1G2_union = compute_mask(data, model='SHG1G2', kind='union')
mask_HG1G2_union = compute_mask(data, model='HG1G2', kind='union')
mask_HG_union = compute_mask(data, model='HG', kind='union')

mask_SHG1G2_inter = compute_mask(data, model='SHG1G2', kind='inter')
mask_HG1G2_inter = compute_mask(data, model='HG1G2', kind='inter')
mask_HG_inter = compute_mask(data, model='HG', kind='inter')

maskFINK_inter = compute_mask(data, R_min=0.305, model='FINK', kind='inter')
maskFINK_union = compute_mask(data, R_min=0.305, model='FINK', kind='union')

for model in ['HG', 'HG1G2', 'SHG1G2', 'FINK']:
    print_statistics(data, model)
    print()

  All data       : 122675  (100.00%)
  Mask HG (g∩r) : 120023  ( 97.84%)
  Mask HG (gUr) : 120024  ( 97.84%)

  All data       : 122675  (100.00%)
  Mask HG1G2 (g∩r) :  47177  ( 38.46%)
  Mask HG1G2 (gUr) :  85363  ( 69.58%)

  All data       : 122675  (100.00%)
  Mask SHG1G2 (g∩r) :  64279  ( 52.40%)
  Mask SHG1G2 (gUr) :  97579  ( 79.54%)

  All data       : 122675  (100.00%)
  Mask FINK (g∩r) :  63092  ( 51.43%)
  Mask FINK (gUr) :  95593  ( 77.92%)



In [14]:
maskFINK = maskFINK_union
mask_HG1G2 = mask_HG1G2_union

## Difference in absolute magnitude: 2x2

In [16]:
# VERSION 2x2 for ATLAS
fig, ax = plt.subplots(
    2,
    2,
    figsize=fs.figsize(0.5),
    sharex=True,
    sharey=True,
    gridspec_kw={
        "hspace": 0.02,
        "wspace": 0.02,
        "top": 0.97,
        "bottom": 0.13,
        "left": 0.1,
        "right": 0.98,
    },
)

r = [-1.8,1.8]
b = 75

# --------------------------------------------------------------------------------
# Histograms
for i, f in enumerate(["g", "r"]):
    
    x = data[f"HG_H_{f}"] - data[f"SHG1G2_H_{f}"]
    ax[i, 0].hist(
        x[maskFINK],
        range=r,
        bins=b,
        label=f"{f}",
        color=fink_colors[i],
        alpha=1,
        histtype="step",
    )

    x = data[f"HG1G2_H_{f}"] - data[f"SHG1G2_H_{f}"]
    ax[i, 1].hist(
        x[maskFINK & mask_HG1G2],
        range=r,
        bins=b,
        label=f"{f}",
        color=fink_colors[i],
        alpha=1,
        histtype="step",
    )
    
    ax[i, 0].axvline(0, ls='--', color='grey')
    ax[i, 1].axvline(0, ls='--', color='grey')

# --------------------------------------------------------------------------------
# Axes
for a in ax.ravel():
    a.set_yscale("log")

ax[0, 0].set_yticks([1, 10, 1e2, 1e3, 1e4])
ax[0, 0].set_yticklabels(["", "$10^1$", "$10^2$", "$10^3$", "$10^4$"])

ax[0, 0].set_ylabel("Count in g")
ax[1, 0].set_ylabel("Count in r")


ax[1, 0].set_xlabel(
    "$\Delta$H ($\\texttt{HG}$ - $\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$)"
)
ax[1, 1].set_xlabel(
    "$\Delta$H ($\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$ - $\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$)"
)



# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "abs_mag.png"),
    facecolor="white",
)

fig.savefig(
   os.path.join(data_fink, "gfx", "article", "abs_mag.pgf"),
)
plt.close()

## Difference in absolute magnitude: 3x2

In [ ]:
# VERSION 3x2 for ATLAS
fig, ax = plt.subplots(
    2,
    3,
    figsize=fs.figsize(0.5),
    sharex=True,
    sharey=True,
    gridspec_kw={
        "hspace": 0.02,
        "wspace": 0.02,
        "top": 0.94,
        "bottom": 0.12,
        "left": 0.1,
        "right": 0.98,
    },
)

r = [-2, 2]
b = 80

# --------------------------------------------------------------------------------
# Histograms
for i, f in enumerate(["g", "r"]):
    x = data[f"HG_H_{f}"] - data[f"sHG1G2_H_{f}"]

    ax[i, 0].hist(
        x[mask_HG1G2],
        range=r,
        bins=b,
        label=f"{f}",
        color=fink_colors[i],
        alpha=1,
        histtype="stepfilled",
    )

    x = data[f"HG1G2_H_{f}"] - data[f"sHG1G2_H_{f}"]
    ax[i, 1].hist(
        x[mask_HG1G2],
        range=r,
        bins=b,
        label=f"{f}",
        color=fink_colors[i],
        alpha=1,
        histtype="stepfilled",
    )

    x = data[f"phase_functions.Misc/Atlas.cyan.H.value"] - data[f"sHG1G2_H_{f}"]
    ax[i, 2].hist(
        x[mask_HG1G2],
        range=r,
        bins=b,
        color="cyan",
        alpha=0.75,
        histtype="stepfilled",
        label="cyan",
    )

    x = data[f"phase_functions.Misc/Atlas.orange.H.value"] - data[f"sHG1G2_H_{f}"]
    ax[i, 2].hist(
        x[mask_HG1G2],
        range=r,
        bins=b,
        color="orange",
        alpha=0.75,
        histtype="stepfilled",
        label="orange",
    )

# --------------------------------------------------------------------------------
# Atlas - ZTF colors
ySun = 5e3
for c in ["orange", "cyan"]:
    ax[0, 2].text(
        skatools.solar_color(f"Misc/Atlas.{c}", "Palomar/ZTF.g"),
        ySun,
        "$\\odot$",
        color=c,
        ha="center",
        va="center",
    )
    ax[1, 2].text(
        skatools.solar_color(f"Misc/Atlas.{c}", "Palomar/ZTF.r"),
        ySun,
        "$\\odot$",
        color=c,
        ha="center",
        va="center",
    )

# --------------------------------------------------------------------------------
# Axes
for a in ax.ravel():
    a.set_yscale("log")


ax[1, 0].set_xlabel("$\Delta$H")
ax[1, 1].set_xlabel("$\Delta$H")
ax[1, 2].set_xlabel("$\Delta$H")

ax[0, 0].set_yticks([1, 10, 1e2, 1e3, 1e4])
ax[0, 0].set_yticklabels(["", "$10^1$", "$10^2$", "$10^3$", "$10^4$"])
ax[0, 0].set_xticks([-2, -1, 0, 1, 2])
ax[0, 0].set_xticklabels(["-2", "-1", "0", "1", ""])

ax[0, 0].set_ylabel("Count in g")
ax[1, 0].set_ylabel("Count in r")

ax[0, 0].text(
    0,
    1.4e4,
    "$\\texttt{HG}$ - $\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    ha="center",
    fontsize="small",
)
ax[0, 1].text(
    0,
    1.4e4,
    "$\\texttt{HG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$ - $\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    ha="center",
    fontsize="small",
)
ax[0, 2].text(
    0,
    1.4e4,
    "Atlas - $\\texttt{sHG}$$_{\\texttt{1}}$$\\texttt{G}$$_{\\texttt{2}}$",
    ha="center",
    fontsize="small",
)

ax[0, 2].legend(loc="lower right", fontsize="xx-small")

# --------------------------------------------------------------------------------
fig.savefig(
    os.path.join(data_fink, "gfx", "article", "abs_mag.png"),
    facecolor="white",
)

fig.savefig(
    os.path.join(data_fink, "gfx", "article", "abs_mag.pgf"),
)
plt.close()